# Alogrithm to adjust trace [v1.0]

In [12]:
# imports
import pdb
from pypit import armsgs
from pypit import pyputils
msgs = pyputils.get_dummy_logger()
armsgs.pypit_logger = msgs
from pypit import ginga
from pypit import arutils as arut
#from xastropy.xutils import xdebug as xdb

## Gaussian profile

In [2]:
def trace_gweight(fimage, xcen, ycen, sigma, invvar=None):
    from scipy.special import erf
    # Setup
    nx = fimage.shape[1]
    ny = fimage.shape[0]
    ncen = xcen.size
    xnew = xcen.copy()
    xerr = 0.0 * xnew # Allocate memory

    if invvar is None:
        invvar = np.ones_like(fimage)

    # More setting up
    #lower = xcen - 3.0*sigma
    #upper = xcen + 3.0*sigma
    x_int = np.round(xcen).astype(int)
    nstep = 2*int(3.0*sigma) - 1
    #i1 = x_int - nstep/2
    #i2 = i1 + nstep - 1

    weight = np.zeros_like(xcen)
    numer  = np.zeros_like(xcen)
    meanvar = np.zeros_like(xcen)
    bad = np.zeros_like(xcen).astype(bool)
    #xtemp = (np.arange(nstep)-nstep/2)/5.  * sigma

    for i in range(nstep): #=0, nstep-1 do begin
        xh = x_int - nstep/2 + i
        xtemp = (xh - xcen - 0.5)/sigma/np.sqrt(2.0)
        g_int = (erf(xtemp+1./sigma/np.sqrt(2.0)) - erf(xtemp))/2.
        try:
            xs = np.minimum(np.maximum(xh,0),(nx-1))
        except:           
            pdb.set_trace()

        cur_weight = fimage[ycen, xs] * (invvar[ycen, xs] > 0) * g_int * ((xh >= 0) & (xh < nx))
        weight += cur_weight
        numer += cur_weight * xh
        meanvar = meanvar + cur_weight * cur_weight * (xcen-xh)**2 / (
                            invvar[ycen, xs] + (invvar[ycen, xs] == 0))
        #pdb.set_trace()
        bad = np.any([bad, xh < 0, xh >= nx], axis=0)
        
    xerr[:] = 999.0
    #pdb.set_trace()
    good = (~bad) & (weight > 0)
    if np.sum(good) > 0:
        xnew[good] = numer[good]/weight[good]
        xerr[good] = np.sqrt(meanvar[good])/weight[good]
    # Return
    return xnew, xerr

### Load image + trace

In [3]:
img_file = '/Users/xavier/Keck/LRIS/data/2016apr06/red_A/Science/spec2d_J0817+1351_LRISr_2016Apr06T060223.fits'
hdu = fits.open(img_file)
names = [hdu[i].name for i in range(len(hdu))]
exten = names.index('DET2-SKYSUB')
skysub = hdu[exten].data
skysub.shape

(2048, 1024)

In [4]:
spec_file = '/Users/xavier/Keck/LRIS/data/2016apr06/red_A/Science/spec1d_J0817+1351_LRISr_2016Apr06T060223.fits'
hdu1 = fits.open(spec_file)
tbl = Table(hdu1[5].data)
trace = tbl['obj_trace']

### Refine

In [19]:
reload(arut)
xnew, xerr = arut.trace_gweight(skysub, trace.data, np.arange(skysub.shape[0]).astype(int), 3.)

In [20]:
xnew2, xerr2 = arutils.trace_gweight(skysub, xnew, np.arange(skysub.shape[0]).astype(int), 3.)

In [21]:
plt.clf()
plt.plot(np.arange(xnew.size), xnew2-trace.data)
plt.show()

### Display

In [8]:
viewer, ch = ginga.show_image(skysub)

In [9]:
ginga.show_trace(viewer, ch, trace.data, 'orig', clear=True, color='blue')
ginga.show_trace(viewer, ch, xnew2, 'x2', color='red')

### Summary
    CRs cause trouble
    Offset is ~0.1 pixels and very hard to see by eye if correct
    Probably not worth the trouble at this stage